In [ ]:
%cd /app


import pandas as pd
import matplotlib.pyplot as plt 

data_df = pd.read_csv("/app/data/final.csv", index_col=[0], parse_dates=True)
data_df = data_df.dropna(subset=['wti_cush_spot'])

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

SEED = 1234

In [ ]:
y_var = "wti_cush_spot"
x_var = [c for c in data_df.columns if c != y_var]

In [ ]:
validation_df = data_df.iloc[-52:]
y_valid, X_valid = validation_df[y_var], validation_df[x_var]
train_df = data_df.iloc[:-52]
y_train, X_train = data_df[y_var], data_df[x_var]

In [ ]:
tscv = TimeSeriesSplit()

models = {
    'random_forest': RandomForestRegressor(
        min_samples_leaf=5,
        random_state=SEED
    )
}

param_grid = {
    "random_forest": {
        "n_estimators": [10,20,30,40,50]
    }
}

results = []
for name, model in models.items():

    grid_search = GridSearchCV(
        estimator = model,
        param_grid = param_grid[name],
        return_train_score=True,
        cv = tscv
    ).fit(X_train, y_train)
    result = {"model": name, "cv_results": pd.DataFrame(grid_search.cv_results_)}
    results.append(result)